# Import

In [1]:
import json
from jovis_model.configs.base import BaseConfig
from jovis_model.datasets.common import CommonDataModule
from jovis_model.models.klue.sequence_classification import SCTransformer

/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
params = {
    "pkg": "klue",
    "task": "ynat",
    "data_dir": "/home/omnious/workspace/jovis/jovis-model/jovis_model/_db/klue/ynat-v1.1",
    "train_file_name": "ynat-v1.1_train.json",
    "dev_file_name": "ynat-v1.1_dev.json",
    "output_dir": "/home/omnious/workspace/jovis/jovis-model/outputs",
}

In [3]:
config = BaseConfig(
    **params
)

In [5]:
dm = CommonDataModule(config)
train_dataloader = dm.train_dataloader()
config.params.num_labels = len(dm.processor.get_labels())
config.params.dataset_size = len(train_dataloader.dataset)

In [6]:
model = SCTransformer(
    config=config,
    metrics={}
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for batch in train_dataloader:
    loss = model.training_step(batch, 0)
    break